In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import torchvision
import torch
from torch import nn
import torch.nn.functional as F
import cv2
import archs
import dataset
import data_augmentation
import skimage.feature
import skimage.filters
import skimage.exposure

In [2]:
a_0 = np.load('inputs/image_train_color_0915/img_002be4f.npy')
a_1 = np.load('inputs/image_train_color_0915/img_0011165.npy')

In [3]:
b_0 = np.load('inputs/mask_train_color_0915/msk_002be4f.npy')
b_1 = np.load('inputs/mask_train_color_0915/msk_0011165.npy')
b_0.shape

In [4]:
torch.Tensor(a_0).size(0)

In [5]:
b_0 = b_0.astype('uint8')

In [6]:
a_0 = a_0/255
a_1 = a_1/255

In [7]:
resize = data_augmentation.Resize()
a_0 = a_0.transpose(2, 0, 1)
a_0 = resize(a_0)
a_0 = a_0.transpose(1, 2, 0)

a_1 = a_1.transpose(2, 0, 1)
a_1 = resize(a_1)
a_1 = a_1.transpose(1, 2, 0)

In [8]:
b_0 = resize(b_0)
b_0 = b_0.transpose(1, 2, 0)

b_1 = resize(b_1)
b_1 = b_1.transpose(1, 2, 0)
b_0.shape

In [9]:
mask = b_1.copy()
mask.shape

In [10]:
image = a_1.copy()
image.shape

In [11]:
image = image.transpose(2, 0, 1)
mask = mask.transpose(2, 0, 1)
image.shape
mask.shape

In [12]:
mask[0,:,:].astype('uint8')

In [17]:
mask.shape

In [26]:
plt.imshow(mask[0])

In [21]:
plt.imshow(a_1)
plt.show()

In [23]:
plt.imshow(mask[1])

In [28]:
x, y, w, h = cv2.boundingRect(b_0[:,:,0])

In [29]:
plt.imshow(a_0[y:y+h, x:x+w,:])

In [33]:
plt.imshow(b_0[:, :, 0])

In [34]:
plt.imshow(np.stack([b_0[:,:,0],b_0[:,:,0],b_0[:,:,0]]).transpose(1,2,0) * a_0)

In [36]:
plt.imshow(np.stack([b_0[:,:,0],b_0[:,:,0],b_0[:,:,0]]).transpose(1,2,0))

In [37]:
left = np.where(b_0[:,:,0].mean(axis=1)>0.2)[0][0]
right = np.where(b_0[:,:,0].mean(axis=1)>0.2)[0][-1]
top = np.where(b_0[:,:,0].mean(axis=0)>0.2)[0][0]
bottom = np.where(b_0[:,:,0].mean(axis=0)>0.2)[0][-1]

In [38]:
plt.imshow(a_0[left:right, top:bottom, :])

In [39]:
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(a_0)
plt.subplot(1,2,2)
plt.imshow(a_1)
plt.show()

In [42]:
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(skimage.exposure.adjust_gamma(a_0, gamma=0.8))
plt.subplot(1,2,2)
plt.imshow(skimage.exposure.adjust_gamma(a_1, gamma=0.8))
plt.show()

In [43]:
plt.figure(figsize=(10,10))
plt.subplot(1,3,1)
plt.imshow(a_0[:, :, 0])
plt.subplot(1,3,2)
plt.imshow(a_0[:, :, 1])
plt.subplot(1,3,3)
plt.imshow(a_0[:, :, 2])
plt.show()

In [44]:
plt.figure(figsize=(10,10))
plt.subplot(1,3,1)
plt.imshow(a_1[:, :, 0])
plt.subplot(1,3,2)
plt.imshow(a_1[:, :, 1])
plt.subplot(1,3,3)
plt.imshow(a_1[:, :, 2])
plt.show()

In [45]:
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(skimage.color.rgb2gray(a_0))
plt.subplot(1,2,2)
plt.imshow(skimage.color.rgb2gray(a_1))
plt.show()

In [46]:
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
adapt_hist_a_0 = skimage.exposure.equalize_adapthist(skimage.color.rgb2gray(a_0))
plt.imshow(adapt_hist_a_0)
plt.subplot(1,2,2)
adapt_hist_a_1 = skimage.exposure.equalize_adapthist(skimage.color.rgb2gray(a_1))
plt.imshow(adapt_hist_a_1)
plt.show()

In [47]:
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
adjust_loss_a_0 = skimage.exposure.adjust_log(skimage.color.rgb2gray(a_0))
plt.imshow(adjust_loss_a_0)
plt.subplot(1,2,2)
adjust_loss_a_1 = skimage.exposure.adjust_log(skimage.color.rgb2gray(a_1))
plt.imshow(adjust_loss_a_1)
plt.show()

In [48]:
def trans_dog(image, sigma_1, sigma2):
    gausian_0 = skimage.filters.gaussian(skimage.color.rgb2gray(image), sigma=sigma_1)
    gausian_1 = skimage.filters.gaussian(skimage.color.rgb2gray(image), sigma=sigma2)
    dog = gausian_0-gausian_1
    dog = dog/dog.max()
    return dog

In [49]:
def load_ben_color(image, sigmaX=10):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
        
    return image

In [51]:
plt.figure(figsize=(12,10))
plt.subplot(1,3,1)
plt.imshow(trans_dog(a_0, 1, 0.8))
plt.subplot(1,3,2)
plt.imshow(trans_dog(a_0, 1, 0.5))
plt.subplot(1,3,3)
plt.imshow(trans_dog(a_0, 1, 0.2))
plt.show()

In [52]:
plt.figure(figsize=(12,10))
plt.subplot(1,3,1)
plt.imshow(trans_dog(a_0, 3, 2))
plt.subplot(1,3,2)
plt.imshow(trans_dog(a_0, 3, 1))
plt.subplot(1,3,3)
plt.imshow(trans_dog(a_0, 3, 0.5))
plt.show()

In [53]:
dog_ = trans_dog(a_0, 1, 0.5)
plt.imshow(skimage.exposure.equalize_adapthist(np.where(dog_ > 0, dog_, 0)))

In [70]:
plt.figure(figsize=(8,8))
a = np.stack([trans_dog(a_0, 1, 0.5), trans_dog(a_0, 1, 0.25),trans_dog(a_0, 1, 0.75)]).transpose(1,2,0)
plt.imshow(a)

In [75]:
plt.figure(figsize=(8,8))
plt.imshow(skimage.exposure.equalize_adapthist(np.where(a > 0, a, 0)))

In [85]:
a_0[:, :, 0] a_0[:, :, 0].mean()

In [55]:
plt.imshow(skimage.exposure.adjust_gamma(skimage.color.rgb2gray(a_0), gamma=0.8))

In [76]:
a_0_0 = skimage.exposure.equalize_adapthist(skimage.exposure.adjust_log(skimage.color.rgb2gray(a_0)))
a_0_1 = skimage.exposure.equalize_adapthist(np.where(dog_ > 0, dog_, 0))
a_0_2 = skimage.exposure.equalize_adapthist(skimage.exposure.adjust_gamma(skimage.color.rgb2gray(a_0), gamma=0.8))

In [77]:
plt.imshow(a_0_1)

In [78]:
plt.imshow(np.stack([a_0_0, a_0_1, a_0_2]).transpose(1,2,0))

In [79]:
plt.figure(figsize=(10,10))
plt.subplot(1,4,1)
plt.imshow(b_0[:, :, 0])
plt.subplot(1,4,2)
plt.imshow(b_0[:, :, 1])
plt.subplot(1,4,3)
plt.imshow(b_0[:, :, 2])
plt.subplot(1,4,4)
plt.imshow(b_0[:, :, 3])
plt.show()

In [80]:
plt.figure(figsize=(10,10))
plt.subplot(1,4,1)
plt.imshow(b_1[:, :, 0])
plt.subplot(1,4,2)
plt.imshow(b_1[:, :, 1])
plt.subplot(1,4,3)
plt.imshow(b_1[:, :, 2])
plt.subplot(1,4,4)
plt.imshow(b_1[:, :, 3])
plt.show()

In [ ]:
model = archs.ResNet18WithUNet(4)
vgg = archs.VGGBlock(in_channels=3, middle_channels=64, out_channels=64)
pool = nn.MaxPool2d(2, 2)

In [ ]:
a_0 = torch.Tensor(a_0[0:256, 0:256])
a_1 = torch.Tensor(a_0[0:256, 0:256])
a_0_0 = a_0.view(1, 3, a_0.shape[0], a_0.shape[1])
a_1_0 = a_1.view(1, 3, a_1.shape[0], a_1.shape[1])

In [ ]:
a_0.shape

In [ ]:
a_0 = a_0[0:256, 0:256].transpose(2,0,1)

In [ ]:
a_0.shape

In [ ]:
random_crop = data_augmentation.RandomCrop()
random_crop(a_0).shape

In [ ]:
horizon = data_augmentation.Horizontal_flip()
horizon(a_0).shape

In [ ]:
vertical = data_augmentation.Vertical_flip()
vertical(a_0).shape

In [ ]:
random_crop = dataset.RandomCrop()
random_crop(b_0).shape

In [ ]:
torch.Tensor(b_0)

In [ ]:
batch = torch.cat([a_0, a_1])

In [ ]:
batch.shape

In [ ]:
vgg(batch).shape

In [ ]:
convrelu(batch).shape

In [ ]:
pool(vgg(batch)).shape

In [ ]:
vgg(pool(batch)).shape

In [ ]:
model_1 = nn.Sequential(
nn.Conv2d(in_channels=3, out_channels = 64, kernel_size=2, stride=2))

In [ ]:
batch.shape

In [ ]:
model_1(batch).shape

In [ ]:
model(batch).shape